In [1]:
%matplotlib inline
import matplotlib.pylab as plt
import numpy as np

from sigvisa import Sigvisa
from sigvisa.graph.sigvisa_graph import SigvisaGraph
from sigvisa.signals.io import fetch_waveform
from sigvisa.source.event import Event

import cPickle as pickle

In [2]:
with open("/home/dmoore/python/sigvisa/logs/mcmc/00598/step_000209/pickle.sg", 'rb') as f:
    sg = pickle.load(f)
    
print sg.current_log_p_breakdown()



n_uatemplate: -24.2
n_event: -182.2
ev priors: ev -95.2
ev observations: ev 0.0
tt_residual: ev -1205.2
ev global cost (n + priors + tt): -1482.5
coda_decay: ev -965.1 ua -4.2 total -969.3
peak_decay: ev -923.3 ua -3.4 total -926.7
peak_offset: ev -978.9 ua -2.6 total -981.5
coda_height: ev -1037.0 ua -4.6 total -1041.5
mult_std_wiggle: ev 298.1 ua 1.7 total 299.8
coef jointgp: 0.0
total param: ev -3606.1 ua -13.1 total -3619.3
priors+params: ev -5088.7 ua -37.3 total -5126.0
station noise (observed signals): -115290.3
noise model prior lp: 40.7
overall: -120375.6
official: -120375.6
None


/home/dmoore/python/sigvisa/models/signal_model.py:587: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  return TransientCombinedSSM(components, TSSM_NOISE_PADDING)


In [5]:
from sigvisa.infer.event_mcmc import ev_phasejump_helper, ev_move_full, phases_changed
from sigvisa.models.distributions import TruncatedGaussian
import copy

In [4]:
old_ev = sg.get_event(5)
new_ev= Event(lon=-110.57, lat=41.37, depth=2.4, time=1239045221.0, mb=3.0)


In [12]:
for i in range(30):
    c1 = i/30.0
    c2 = 1.0-c1
    int_ev = Event(lon=c1*new_ev.lon + c2*old_ev.lon, 
                   lat=c1*new_ev.lat + c2*old_ev.lat,
                   depth=c1*new_ev.depth + c2*old_ev.depth,
                   time=c1*new_ev.time + c2*old_ev.time,
                   mb=c1*new_ev.mb + c2*old_ev.mb)
    print int_ev
    
    print phases_changed(sg, 5, int_ev)

evid None, loc 108.33 W 41.84 N, depth 4.5km, time 1239045222.3, mb 3.0, natural source
({'PDAR': set([]), 'YKA': set([]), 'NVAR': set([]), 'PFO': set([]), 'ANMO': set([]), 'ULM': set([]), 'NEW': set([]), 'ILAR': set([])}, {'PDAR': set([]), 'YKA': set([]), 'NVAR': set([]), 'PFO': set([]), 'ANMO': set([]), 'ULM': set([]), 'NEW': set([]), 'ILAR': set([])}, False)
evid None, loc 108.40 W 41.83 N, depth 4.4km, time 1239045222.3, mb 3.0, natural source
({'PDAR': set([]), 'YKA': set([]), 'NVAR': set([]), 'PFO': set([]), 'ANMO': set([]), 'ULM': set([]), 'NEW': set([]), 'ILAR': set([])}, {'PDAR': set([]), 'YKA': set([]), 'NVAR': set([]), 'PFO': set([]), 'ANMO': set([]), 'ULM': set([]), 'NEW': set([]), 'ILAR': set([])}, False)
evid None, loc 108.48 W 41.81 N, depth 4.4km, time 1239045222.3, mb 3.0, natural source
({'PDAR': set([]), 'YKA': set([]), 'NVAR': set([]), 'PFO': set([]), 'ANMO': set([]), 'ULM': set([]), 'NEW': set([]), 'ILAR': set([])}, {'PDAR': set([]), 'YKA': set([]), 'NVAR': set([])

In [5]:
n = sg.evnodes[5]["loc"]
params = ["lon", "lat", "depth", "time", "mb"]
std = 100.0
np.random.seed(11)
lpo = sg.current_log_p()
lp_old, lp_new, log_qforward, log_qbackward, replicate_forward = ev_move_full(sg, n, std, params, 
                                                                              force_proposed_ev=new_ev,
                                                                              return_debug=True)
lpn = sg.current_log_p()

print
print  "sanity check should be zero:", lpn-lpo

print "results"
print lp_new, lp_old, log_qforward, log_qbackward
print lp_new- lp_old + log_qbackward - log_qforward

decouple False at PD31 with p_decoupled 0.000 lqf -0.000 heuristic False
decouple False at YKR8 with p_decoupled 0.000 lqf -0.000 heuristic None
decouple True at NV01 with p_decoupled 1.000 lqf -0.000 heuristic True
decouple True at PFO with p_decoupled 1.000 lqf -0.000 heuristic True
decouple False at ANMO with p_decoupled 0.000 lqf -0.000 heuristic None
decouple True at ULM with p_decoupled 1.000 lqf -0.000 heuristic None
decouple False at NEW with p_decoupled 0.000 lqf -0.000 heuristic False
decouple False at IL31 with p_decoupled 0.000 lqf -0.000 heuristic None
using assoc [('Pg', None)] with lp 0.0
dumb logq mult_wiggle_std NV01 Pg 0.603456165192 0.516744614107  unfixed
dumb logq amp_transfer NV01 Pg 4.99478425845 -1.54455346898  unfixed
dumb logq coda_decay NV01 Pg -2.35661718881 -1.71911668046  unfixed
dumb logq peak_offset NV01 Pg 1.18399333534 -1.3924949001  unfixed
dumb logq tt_residual NV01 Pg 0.73370124716 -1.69240827272  unfixed
dumb logq peak_decay NV01 Pg -2.8705254676 -

In [6]:
replicate_forward()

In [9]:
sg.debug_dump("/home/dmoore/python/sigvisa/logs/mcmc/00598/step_000208/", pickle_only=True)

saving debug dump to /home/dmoore/python/sigvisa/logs/mcmc/00598/step_000208/...
saved pickled graph


In [7]:
sg.current_log_p()

-120374.19435224481

In [8]:
print lpn

-120375.557449
